# Persistent BabyAGI

This notebook showcases [PersistentBabyAGI](../../../../slangchain/autonomous_agents/baby_agi.py) that is build on top of [BabyAgi](https://github.com/hwchase17/langchain/blob/master/docs/use_cases/autonomous_agents/baby_agi_with_agent.ipynb).

In a nutshell, PersistentBabyAGI extends the functionality for BabyAGI, allowing you to persist the task and task results in the form of Vectorstores. You can pick up where you left off by saving/loading the task and task results Vectorstores.

## Install and Import Required Modules

In [20]:
import os
from typing import Optional
from langchain import PromptTemplate

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [2]:
%pip install faiss-cpu > /dev/null
%pip install google-search-results > /dev/null
from langchain.vectorstores import FAISS


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Persisting BabyAGI


Don't forget to set the OpenAI API key.

In [3]:
os.environ["OPENAI_API_KEY"] = ""

### Define the PersistBabyAGI Controller

Define the tools and agents for the controller

In [4]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, LLMChain
from langchain.agents import load_tools

llm = OpenAI(temperature=0)

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=llm, prompt=todo_prompt)
tools = load_tools(tool_names=["ddg-search"])
tools.append(
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ))


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""

suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

agent = ZeroShotAgent.from_llm_and_tools(
    llm=llm,
    tools=tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
    verbose=True)

# Instantiate agent executor
execution_chain = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True)

### Initial Run - PersistBabyAGI

Now it's time to create the PersistBabyAGI controller and watch it try to accomplish your objective.

In [5]:
OBJECTIVE = "Write a weather report for SF today"

In [6]:
llm = OpenAI(temperature=0)

In [7]:
from slangchain.autonomous_agents.baby_agi import PersistentBabyAGI

# Logging of LLMChains
verbose = True
# If None, will keep on going forever
max_iterations: Optional[int] = 2
baby_agi = PersistentBabyAGI.from_llm_and_vectorstores(
    llm=llm, execution_chain=execution_chain, verbose=verbose, max_iterations=max_iterations
)

NOTE: Redirects are currently not supported in Windows or MacOs.


- Run BabyAGI for 2 iterations.

In [8]:
baby_agi({"objective": OBJECTIVE})



> Entering new AgentExecutor chain...
Thought: I need to create a todo list
Action: TODO
Action Input: Create a todo list for writing a weather report for SF today
Observation: 

1. Research current weather conditions in SF
2. Gather data on temperature, humidity, wind speed, and other relevant weather conditions
3. Research any upcoming weather changes
4. Write a brief introduction to the weather report
5. Describe current weather conditions in SF
6. Discuss any upcoming weather changes
7. Summarize the weather report
8. Proofread and edit the report
9. Publish the report
Thought: I now know the final answer
Final Answer: A todo list for writing a weather report for SF today includes: 1. Research current weather conditions in SF, 2. Gather data on temperature, humidity, wind speed, and other relevant weather conditions, 3. Research any upcoming weather changes, 4. Write a brief introduction to the weather report, 5. Describe current weather conditions in SF, 6. Discuss any upcoming 

{'objective': 'Write a weather report for SF today'}

### Results

- Results are stored in the PersistentBabyAGI's results variable

In [9]:
for task_result in baby_agi.results:
    print(f"Task Name: {task_result.task_name}\n")
    print(f"Result: {task_result.result}\n\n")

Task Name: 1: Make a todo list

Result: A todo list for writing a weather report for SF today includes: 1. Research current weather conditions in SF, 2. Gather data on temperature, humidity, wind speed, and other relevant weather conditions, 3. Research any upcoming weather changes, 4. Write a brief introduction to the weather report, 5. Describe current weather conditions in SF, 6. Discuss any upcoming weather changes, 7. Summarize the weather report, 8. Proofread and edit the report, and 9. Publish the report.


Task Name: 2: Gather data on precipitation, cloud cover, and other relevant weather conditions

Result: Today in San Francisco, temperatures will be in the mid to upper 40s. Tomorrow will be partly cloudy to mostly sunny, with highs ranging from upper 50s at the coast to mid 60s near the bay to around 70 degrees inland. Southwest winds will be 5 to 15 mph. Friday night will be mostly cloudy with highs in the mid 50s to the lower 60s. There is a chance of up to half an inch of

### Persisting Task and Task Results via Vector Stores

- Tasks and Task Results persist as Vector Stores in the instantiated PersistBabyAGI class

In [10]:
def print_faiss_index(vectorstore):
    num_vectors = vectorstore.index.ntotal
    from colorama import Fore

    # Loop through all the vectors and print them
    for i in range(num_vectors):
        vector = vectorstore.index.reconstruct(i)
        doc = vectorstore.similarity_search_by_vector(vector, k=1)
        header = f"{doc[0].metadata['task_id']}: {doc[0].metadata['task_name']}" if "task_id" in doc[0].metadata.keys() else f"{doc[0].metadata}"
        print(Fore.GREEN + f"{header}\n")
        print(Fore.WHITE + f"{doc[0].page_content}\n")

#### Task Vector Store

- The task_vectorstore Vector Store contains the task that have yet to be completed. Tasks 1 and 2 have been completed during the initial run.

In [11]:
print_faiss_index(baby_agi.task_vectorstore)

3: Analyze the data to determine any upcoming weather changes

Analyze the data to determine any upcoming weather changes

4: Research current weather forecasts for SF

Research current weather forecasts for SF

5: Compare the current weather conditions to historical averages for San Francisco.

Compare the current weather conditions to historical averages for San Francisco.

6: Identify any potential hazards associated with the forecasted weather.

Identify any potential hazards associated with the forecasted weather.

7: Research any potential impacts of the forecasted weather on local businesses and activities.

Research any potential impacts of the forecasted weather on local businesses and activities.

8: Identify any potential health risks associated with the forecasted weather.

Identify any potential health risks associated with the forecasted weather.

9: Research any potential impacts of the forecasted weather on local wildlife.

Research any potential impacts of the forecast

#### Task Results Vector Store

- The task_results Vector Store contains the task that have been completed (tasks 1 and 2).

In [12]:
print_faiss_index(baby_agi.task_result_vectorstore)

1: Make a todo list

A todo list for writing a weather report for SF today includes: 1. Research current weather conditions in SF, 2. Gather data on temperature, humidity, wind speed, and other relevant weather conditions, 3. Research any upcoming weather changes, 4. Write a brief introduction to the weather report, 5. Describe current weather conditions in SF, 6. Discuss any upcoming weather changes, 7. Summarize the weather report, 8. Proofread and edit the report, and 9. Publish the report.

2: Gather data on precipitation, cloud cover, and other relevant weather conditions

Today in San Francisco, temperatures will be in the mid to upper 40s. Tomorrow will be partly cloudy to mostly sunny, with highs ranging from upper 50s at the coast to mid 60s near the bay to around 70 degrees inland. Southwest winds will be 5 to 15 mph. Friday night will be mostly cloudy with highs in the mid 50s to the lower 60s. There is a chance of up to half an inch of rain possible along parts of the coast

#### Saving the Vector Stores
- The task and task results Vectorstores can be saved to a local directory.

In [13]:
task_vectorstore_dir = "./task_vectorstore"
task_results_vectorstore_dir = "./task_results_vectorstore"

baby_agi.task_vectorstore.save_local(task_vectorstore_dir)
baby_agi.task_result_vectorstore.save_local(task_results_vectorstore_dir)

### Pick up where you left off

PersistBabyAGI controller provides you the capability to continue running from where it left off.

#### Loading Vector Stores

- First load the saved task and task results Vectorstores.

In [14]:
task_vectorstore = FAISS.load_local(task_vectorstore_dir, baby_agi.task_embeddings)
task_results_vectorstore = FAISS.load_local(task_results_vectorstore_dir, baby_agi.task_embeddings)

#### Class Instantiation

- Instantiate PersistBabyAGI with the Vector Stores

In [15]:
persistent_baby_agi = PersistentBabyAGI.from_llm_and_vectorstores(
    llm=llm,
    execution_chain=execution_chain,
    verbose=verbose,
    max_iterations=max_iterations,
    task_embeddings=baby_agi.task_embeddings,
    task_vectorstore=task_vectorstore,
    task_result_vectorstore=task_results_vectorstore
)

### Follo Up Run - PersistBabyAGI

- And continue on your merry way from where the controller left off

In [16]:
persistent_baby_agi({"objective": OBJECTIVE})



> Entering new AgentExecutor chain...
Thought: I need to analyze the data to determine any upcoming weather changes
Action: DuckDuckGo Search
Action Input: SF weather forecast
Observation: San Francisco Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the San Francisco area. Monday evening First Alert Weather forecast - 5/8/23. with Paul Heggen. 4H ago. 03:26. AccuWeather Forecast: Partly cloudy morning, brighter and pleasant afternoon ... San Francisco 61. Oakland 64. San Jose 67. Concord 69. ... Weather Radar: LiveDoppler7 radar images. The National Weather Service forecast for Monday for the greater San Francisco Bay Area calls for a chance of rain in the morning, before becoming sunny with highs in the 50s and 60s. San Francisco, CA - Weather forecast from Theweather.com. Weather conditions with updates on temperature, humidity, wind speed, snow, pressure, etc. for San Francisco ...
Though

{'objective': 'Write a weather report for SF today'}

### PersistBabyAGI - New Task Results

- BabyAGI continued to action and completed task 3 and 4 (from the previous run of tasks 1 and 2)

In [17]:
for task_result in persistent_baby_agi.results:
    print(f"Task Name: {task_result.task_name}\n")
    print(f"Result: {task_result.result}\n\n")

Task Name: 3: Analyze the data to determine any upcoming weather changes

Result: The weather in San Francisco today will be partly cloudy in the morning, becoming sunny with highs in the 50s and 60s. There is a chance of rain in the morning.


Task Name: 4: Research current weather forecasts for SF

Result: The weather in San Francisco today will be partly cloudy in the morning, with a brighter and pleasant afternoon. Temperatures will range from 61 degrees in San Francisco to 69 degrees in Concord. There is no precipitation forecasted.




### PersistBabyAGI - Updated Task Results Vector Store

- The controller had also updated the task results vector store with newly completed tasks 3 and 4.

In [18]:
print_faiss_index(persistent_baby_agi.task_result_vectorstore)

1: Make a todo list

A todo list for writing a weather report for SF today includes: 1. Research current weather conditions in SF, 2. Gather data on temperature, humidity, wind speed, and other relevant weather conditions, 3. Research any upcoming weather changes, 4. Write a brief introduction to the weather report, 5. Describe current weather conditions in SF, 6. Discuss any upcoming weather changes, 7. Summarize the weather report, 8. Proofread and edit the report, and 9. Publish the report.

2: Gather data on precipitation, cloud cover, and other relevant weather conditions

Today in San Francisco, temperatures will be in the mid to upper 40s. Tomorrow will be partly cloudy to mostly sunny, with highs ranging from upper 50s at the coast to mid 60s near the bay to around 70 degrees inland. Southwest winds will be 5 to 15 mph. Friday night will be mostly cloudy with highs in the mid 50s to the lower 60s. There is a chance of up to half an inch of rain possible along parts of the coast

### PersistBabyAGI - Updated Task Vector Store

- And removed completed tasks 3 and 4 from the task Vector Store

In [19]:
print_faiss_index(persistent_baby_agi.task_vectorstore)

5: Compare the current weather conditions to historical averages for San Francisco.

Compare the current weather conditions to historical averages for San Francisco.

6: Identify any potential hazards associated with the forecasted weather.

Identify any potential hazards associated with the forecasted weather.

7: Research any potential impacts of the forecasted weather on local businesses and activities.

Research any potential impacts of the forecasted weather on local businesses and activities.

8: Identify any potential health risks associated with the forecasted weather.

Identify any potential health risks associated with the forecasted weather.

9: Research any potential impacts of the forecasted weather on local wildlife.

Research any potential impacts of the forecasted weather on local wildlife.

10: Identify any potential impacts of the forecasted weather on air quality.

Identify any potential impacts of the forecasted weather on air quality.

11: Research any potential im